# Compare Skyfield and JPL RA/DEC Calculations

In [1]:
import numpy as np
import skyfield
from skyfield.api import load
from skyfield.positionlib import ICRF, Barycentric
import astropy.time
from astropy.units import deg, au, km, meter, day, minute, second
from astropy.coordinates import SkyCoord, ICRS, GCRS, BarycentricMeanEcliptic, HeliocentricMeanEcliptic

# MSE imports
from astro_utils import mjd_to_jd, radec2dir, qv2radec, qvrel2radec, radec_diff
from typing import Tuple

In [2]:
# Browse available JPL ephemeris files
!ls ../data/jpl/ephemeris/*.bsp

../data/jpl/ephemeris/de405.bsp		../data/jpl/ephemeris/de432s.bsp
../data/jpl/ephemeris/de430.bsp		../data/jpl/ephemeris/de435.bsp
../data/jpl/ephemeris/de431.bsp		../data/jpl/ephemeris/de438.bsp
../data/jpl/ephemeris/de431_part-1.bsp	../data/jpl/ephemeris/mar097.bsp
../data/jpl/ephemeris/de431_part-2.bsp


### Position and Observation of Earth and Mars according to Skyfield

In [3]:
# Get earth and mars loaded following Skyfield tutorial
# planets = load('de421.bsp')
# earth, mars = planets['earth'], planets['mars']

# Manually load planetary positions using de431
# JPL Horizons web interface says source for Earth is de431mx (at top of Results box, Ephemeris section)
# Source for Mars is mar097
# Can browse at https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/
planets = load('../data/jpl/ephemeris/de431.bsp')
earth = planets['earth']
mars = planets['mars barycenter']

# load timescale
ts = load.timescale()

# set observation time
obstime_mjd = 58600.0
obstime_jd = mjd_to_jd(obstime_mjd)
obstime_sf = ts.tt_jd(obstime_jd)
t = obstime_sf
# also get obs time in astropy
obstime_ap = astropy.time.Time(obstime_mjd, format='mjd')
# time conversions via astropy are inconsistent :(
# must convert directly from a jd

# observe mars from earth
obs = earth.at(t).observe(mars)
ra, dec, distance = obs.radec()

# report results
print('observe mars from earth with de431 ephemerides')
print(f'ra  : {ra._degrees:10.6f} degrees')
print(f'dec : {dec._degrees:10.6f} degrees')
print(f'dist: {distance.au:10.6f} au')

observe mars from earth with de431 ephemerides
ra  :  76.106846 degrees
dec :  23.884830 degrees
dist:   2.211754 au


In [4]:
# Get position at time t
pos_earth = earth.at(t)
pos_mars = mars.at(t)

***Browse classes used by SkyField***

In [5]:
print(f'earth.center = {mars.center}')
print(f'earth.center_name = {mars.center_name}')
print(f'earth.target = {earth.target}')
print(f'earth.target_name = {earth.target_name}')
print(f'earth.ephemeris = \n', earth.ephemeris)
print(f'earth.positives =', earth.positives)
print(f'earth.negatives =', earth.negatives)
# these are old
# earth.topos('Boston')
# print(f'earth.geometry_of(mars)=', earth.geometry_of(mars))

earth.center = 0
earth.center_name = 0 SOLAR SYSTEM BARYCENTER
earth.target = 399
earth.target_name = 399 EARTH
earth.ephemeris = 
 SPICE kernel file 'de431.bsp' has 14 segments
  JD 1721425.50 - JD 8000002.50  (0-12-31 through 17191-02-28)
      2 -> 299  VENUS BARYCENTER -> VENUS
      1 -> 199  MERCURY BARYCENTER -> MERCURY
      3 -> 399  EARTH BARYCENTER -> EARTH
      3 -> 301  EARTH BARYCENTER -> MOON
      0 -> 10   SOLAR SYSTEM BARYCENTER -> SUN
      0 -> 9    SOLAR SYSTEM BARYCENTER -> PLUTO BARYCENTER
      0 -> 8    SOLAR SYSTEM BARYCENTER -> NEPTUNE BARYCENTER
      0 -> 7    SOLAR SYSTEM BARYCENTER -> URANUS BARYCENTER
      0 -> 6    SOLAR SYSTEM BARYCENTER -> SATURN BARYCENTER
      0 -> 5    SOLAR SYSTEM BARYCENTER -> JUPITER BARYCENTER
      0 -> 4    SOLAR SYSTEM BARYCENTER -> MARS BARYCENTER
      0 -> 3    SOLAR SYSTEM BARYCENTER -> EARTH BARYCENTER
      0 -> 2    SOLAR SYSTEM BARYCENTER -> VENUS BARYCENTER
      0 -> 1    SOLAR SYSTEM BARYCENTER -> MERCURY BARYC

In [6]:
# These are in the ICRS frame (aligned to earth's equator)
print('\nEarth in the ICRS Frame:')
print(f'pos_earth.cirs_xyz(t)         = ', pos_earth.cirs_xyz(t))
print(f'pos_earth.position            = ', pos_earth.position)
print(f'pos_earth.velocity            = ', pos_earth.velocity)

# These are in the ecliptic frame
print('\nEarth in the Ecliptic Frame:')
print(f'pos_earth.ecliptic_xyz()      = ', pos_earth.ecliptic_xyz())
print(f'pos_earth.ecliptic_position() = ', pos_earth.ecliptic_position())
print(f'pos_earth.ecliptic_velocity() = ', pos_earth.ecliptic_velocity())

# Summary attributes
print(f'\nOther attributes of pos_earth class:')
print(f'pos_earth.center = {pos_earth.center}')
print(f'pos_earth.target =', pos_earth.target)
# Magnitudes
print(f'pos_earth.distance() =', pos_earth.distance())
print(f'pos_earth.speed() =', pos_earth.speed())
print(f'pos_earth.t =', pos_earth.t)
print(f'pos_earth.cirs_radec(t) =', pos_earth.cirs_radec(t))
print(f'pos_earth.observer_data =', pos_earth.observer_data)
print(f'pos_earth.observe(mars) =', pos_earth.observe(mars))
print(f'pos_earth.separation_from(mars.at(t)) =', pos_earth.separation_from(mars.at(t)))
print(f'pos_earth.message =', pos_earth.message)


Earth in the ICRS Frame:
pos_earth.cirs_xyz(t)         =  [-0.81335333 -0.53834618 -0.23489244] au
pos_earth.position            =  [-0.81378506 -0.53834165 -0.23340276] au
pos_earth.velocity            =  [ 0.00986741 -0.01280032 -0.00554809] au/day

Earth in the Ecliptic Frame:
pos_earth.ecliptic_xyz()      =  [-8.13785065e-01 -5.86761093e-01 -2.83704743e-06] au
pos_earth.ecliptic_position() =  [-8.13785065e-01 -5.86761093e-01 -2.83704743e-06] au
pos_earth.ecliptic_velocity() =  [ 9.86740640e-03 -1.39509631e-02  1.40419708e-06] au/day

Other attributes of pos_earth class:
pos_earth.center = 0
pos_earth.target = 399
pos_earth.distance() = 1.00326 au
pos_earth.speed() = 0.0170879 au/day
pos_earth.t = <Time tt=2458600.5>
pos_earth.cirs_radec(t) = (<Angle 14h 13m 59.99s>, <Angle -13deg 32' 25.0">, <Distance 1.00326 au>)
pos_earth.observer_data = <skyfield.vectorlib.ObserverData object at 0x7fb16986add0>
pos_earth.observe(mars) = <Astrometric ICRS position and velocity at date t center=3

### Position and Observation of Earth and Mars according to JPL

In [7]:
# JPL: Cartesian coordinates of Earth geocenter in barycentric frame
q_earth_jpl = [-8.137850649885880E-01, -5.867610927308373E-01, -2.837047450366285E-06] * au
v_earth_jpl = [ 9.867406393541108E-03, -1.395096307916507E-02,  1.404197076481182E-06] * au / day

# JPL: Cartesian coordinates of Mars in barycentric frame
q_mars_jpl = [-3.283646374478960E-01, 1.570623707113252E+00, 4.073331592627085E-02] * au
v_mars_jpl = [-1.317734697685353E-02, -1.672780021837941E-03, 2.882733450987910E-04] * au / day

# Cartesian coordinates earth to mars
q_e2m_jpl = q_mars_jpl - q_earth_jpl
v_e2m_jpl = v_mars_jpl - v_earth_jpl

In [8]:
# JPL: Observation from Earth of Mars
ra_jpl = 76.107414227
dec_jpl = 23.884882701
ra_app_jpl = 76.391533106
dec_app_jpl = 23.908809827
delta_jpl = 2.21175980433254 * au
delta_dot_jpl = 11.9879558 * km / second
light_time_jpl = 18.39464538 * minute

### Compare Skyfield vs. JPL 

In [9]:
# difference in position between skyfield and JPL
dq_earth = pos_earth.ecliptic_position().au - q_earth_jpl.value
dq_mars = pos_mars.ecliptic_position().au - q_mars_jpl.value
dq_e2m = dq_earth - dq_mars

# display norms
ndq_earth = np.linalg.norm(dq_earth)
ndq_mars = np.linalg.norm(dq_mars)
ndq_e2m = np.linalg.norm(dq_e2m)

# report
print(f'Difference: Skyfield - JPL')
print(f'Earth:', dq_earth)
print(f'Mars :', dq_mars)
print(f'E2M  :', dq_e2m)
print('\nDistance: Skyfield - JPL')
print(f'Earth: {ndq_earth:5.3e}', )
print(f'Mars : {ndq_mars:5.3e}', )
print(f'E2M  : {ndq_e2m:5.3e}', )

Difference: Skyfield - JPL
Earth: [ 1.74264714e-10 -2.46382914e-10  2.48805459e-14]
Mars : [-2.32302066e-10 -6.34386321e-10 -5.21031204e-11]
E2M  : [4.06566780e-10 3.88003407e-10 5.21280010e-11]

Distance: Skyfield - JPL
Earth: 3.018e-10
Mars : 6.776e-10
E2M  : 5.644e-10


**Conclusion: Position<br>
Skyfield agrees with JPL to 1E-10 AU**<br>
This is essentially "perfect" for practical purposes.

In [10]:
# observe mars from earth
obs = earth.at(t).observe(mars)
ra_sf, dec_sf, distance_sf = obs.radec()

# Extract degrees
ra_sf = ra_sf._degrees
dec_sf = dec_sf._degrees

# Report results
print(f'Difference in astromentric RA/DEC:\n')
diff_sf_sec = \
radec_diff(name1='JPL', name2='Skyfield', ra1=ra_jpl, dec1=dec_jpl, ra2=ra_sf, dec2=dec_sf, 
           obstime_mjd=obstime_mjd, verbose=True)

Difference in astromentric RA/DEC:

Difference in RA: Skyfield - JPL
Skyfield    : 76.106846 deg
JPL         : 76.107414 deg
Diff        : -0.000568 deg (-2.05 seconds)

Difference in DEC: Skyfield - JPL
Skyfield    : 23.884830 deg
JPL         : 23.884883 deg
Diff        : -0.000053 deg (-0.19 seconds)

Unit direction vectors:
Skyfield    : [0.21954911 0.97542761 0.01841651]
JPL         : [0.21954022 0.97542961 0.01841657]
Diff        : [ 8.89319144e-06 -2.00054756e-06 -5.76355302e-08]
AngleDiff   : 1.880 seconds


**Conclusion: Astrometric RA/DEC<br>
Skyfield agrees with JPL to 1.9 arc seconds**<br>
This is quite good, and almost certainly good enough for the intended purpose.

In [11]:
# apparent observation of mars from earth (includes corrections)
obs_app = obs.apparent()
ra_app_sf, dec_app_sf, _ = obs_app.radec()

# Extract degrees
ra_app_sf = ra_app_sf._degrees
dec_app_sf = dec_app_sf._degrees

# Report results
print(f'Difference in apparent RA/DEC: Skyfield vs. JPL\n')
diff_sf_app_sec = \
radec_diff(name1='JPL', name2='Skyfield', ra1=ra_app_jpl, dec1=dec_app_jpl, ra2=ra_app_sf, dec2=dec_app_sf, 
           obstime_mjd=obstime_mjd, verbose=True)

Difference in apparent RA/DEC: Skyfield vs. JPL

Difference in RA: Skyfield - JPL
Skyfield    : 76.102269 deg
JPL         : 76.391533 deg
Diff        : -0.289264 deg (-1041.35 seconds)

Difference in DEC: Skyfield - JPL
Skyfield    : 23.884498 deg
JPL         : 23.908810 deg
Diff        : -0.024311 deg (-87.52 seconds)

Unit direction vectors:
Skyfield    : [0.21962058 0.9754115  0.01841773]
JPL         : [0.21509623 0.97641944 0.01840357]
Diff        : [ 4.52435396e-03 -1.00794297e-03  1.41634615e-05]
AngleDiff   : 956.098 seconds


**Conclusion: apparent RA/DEC:<br>
Skyfield does not agree with JPL on correcting astrometric observations.<br>
Difference is 956 seconds.**<br>
This difference is so large that Skyfield is probably doing a different calculation from JPL.

### RA and DEC in Astropy

In [12]:
# extract degrees
# ra_ap = obs_ap_gcrs.ra.deg
# dec_ap = obs_ap_gcrs.dec.deg

# Get RA and DEC using qv2radec function; expects q, v in barycentric frame
ra, dec, r = qv2radec(q=q_mars_jpl, v=v_mars_jpl, mjd=obstime_mjd, 
                      frame=BarycentricMeanEcliptic, light_lag=False)

# Report results
print(f'Difference in astromentric RA/DEC: Astropy (GCRS) vs. JPL\n')
diff_sf_sec = \
radec_diff(name1='JPL', name2='Astropy', ra1=ra_jpl, dec1=dec_jpl, ra2=ra, dec2=dec, obstime_mjd=obstime_mjd, verbose=True)

Difference in astromentric RA/DEC: Astropy (GCRS) vs. JPL

Difference in RA: Astropy - JPL
Astropy     : 76.107043 deg
JPL         : 76.107414 deg
Diff        : -0.000371 deg (-1.33 seconds)

Difference in DEC: Astropy - JPL
Astropy     : 23.885028 deg
JPL         : 23.884883 deg
Diff        : 0.000145 deg ( 0.52 seconds)

Unit direction vectors:
Astropy     : [0.21954572 0.97542831 0.01841966]
JPL         : [0.21954022 0.97542961 0.01841657]
Diff        : [ 5.49692986e-06 -1.29559449e-06  3.09196065e-06]
AngleDiff   : 1.328 seconds


In [13]:
# Get RA and DEC using qv2radec function; expects q, v in barycentric frame
ra, dec, r = qvrel2radec(q_body=q_mars_jpl, v_body=v_mars_jpl, 
                         q_earth=q_earth_jpl, v_earth=v_earth_jpl,
                         mjd=obstime_mjd, 
                         frame=BarycentricMeanEcliptic)

# Report results
print(f'Difference in astrometric RA/DEC: Astropy (GCRS) vs. JPL\n')
diff_sf_sec = \
radec_diff(name1='JPL', name2='Astropy', ra1=ra_jpl, dec1=dec_jpl, ra2=ra, dec2=dec, obstime_mjd=obstime_mjd, verbose=True)

Difference in astrometric RA/DEC: Astropy (GCRS) vs. JPL

Difference in RA: Astropy - JPL
Astropy     : 76.106756 deg
JPL         : 76.107414 deg
Diff        : -0.000659 deg (-2.37 seconds)

Difference in DEC: Astropy - JPL
Astropy     : 23.885007 deg
JPL         : 23.884883 deg
Diff        : 0.000125 deg ( 0.45 seconds)

Unit direction vectors:
Astropy     : [0.21955021 0.9754273  0.01841974]
JPL         : [0.21954022 0.97542961 0.01841657]
Diff        : [ 9.98988057e-06 -2.30834483e-06  3.17049959e-06]
AngleDiff   : 2.214 seconds


### Demonstrate Affine invariance

In [16]:
# Demonstrate affine invariance
q_eps = np.random.normal(loc=0.0, scale=0.1, size=3) * au
v_eps = np.random.normal(loc=0.0, scale=2.5e5, size=3) * km / second

# Get RA and DEC using qv2radec function; expects q, v in barycentric frame
ra, dec, r = qvrel2radec(q_body=q_mars_jpl+q_eps, v_body=v_mars_jpl+v_eps, 
                         q_earth=q_earth_jpl+q_eps, v_earth=v_earth_jpl+v_eps,
                         mjd=obstime_mjd, frame=BarycentricMeanEcliptic)

# Report results
print(f'Difference in astromentric RA/DEC: Astropy (GCRS) vs. JPL\n')
diff_sf_sec = \
radec_diff(name1='JPL', name2='Astropy', ra1=ra_jpl, dec1=dec_jpl, ra2=ra, dec2=dec, obstime_mjd=obstime_mjd, verbose=True)

Difference in astromentric RA/DEC: Astropy (GCRS) vs. JPL

Difference in RA: Astropy - JPL
Astropy     : 76.106756 deg
JPL         : 76.107414 deg
Diff        : -0.000659 deg (-2.37 seconds)

Difference in DEC: Astropy - JPL
Astropy     : 23.885007 deg
JPL         : 23.884883 deg
Diff        : 0.000125 deg ( 0.45 seconds)

Unit direction vectors:
Astropy     : [0.21955021 0.9754273  0.01841974]
JPL         : [0.21954022 0.97542961 0.01841657]
Diff        : [ 9.98988057e-06 -2.30834483e-06  3.17049959e-06]
AngleDiff   : 2.214 seconds


In [17]:
q_body = q_mars_jpl + q_eps
v_body = v_mars_jpl
q_earth = q_earth_jpl + q_eps
v_earth = v_earth_jpl
mjd = obstime_mjd
frame = BarycentricMeanEcliptic

**Prototype of qvrel2radec**

In [ ]:
zero_au = 0.0 * au
zero_km_sec = 0.0 * km / second

In [ ]:
# Represent the earth in the GCRS frame.  Easy because it has position & velocity zero!
earth_gcrs = SkyCoord(x=zero_au, y=zero_au, z=zero_au, v_x=zero_km_sec, v_y=zero_km_sec, v_z=zero_km_sec,
                      representation_type='cartesian', obstime=obstime_ap, frame=GCRS)
# Represent earth in BarycentricMeanEcliptic frame, using Cartesian representation
earth_bme = earth_gcrs.transform_to(BarycentricMeanEcliptic)
earth_bme.representation_type = 'cartesian'
earth_bme.differential_type = 'cartesian'

In [ ]:
earth_bme.velocity.d_xyz

In [ ]:
v_earth_jpl

In [ ]:
# Correction factor to add to position of earth in BME so it matches JPL
dq = q_earth_jpl - earth_bme.cartesian.xyz
# Correction factor to add to velocity of earth in BME so it matches JPL
dv = v_earth_jpl - earth_bme.velocity.d_xyz

In [ ]:
# Generate the corrected position and velocity of Mars
x, y, z = q_mars_jpl + dq
v_x, v_y, v_z = v_mars_jpl + dv
# Observe Mars in the BarcycentricMeanEcliptic frame
obs_e2m_bme = SkyCoord(x=x, y=y, z=z, v_x=v_x, v_y=v_y, v_z=v_z, obstime=obstime_ap,
                       representation_type='cartesian', frame=BarycentricMeanEcliptic)
# Transform observation to GCRS
obs_ap_gcrs2 = obs_e2m_bme.transform_to(GCRS)

In [ ]:
obs_e2m_bme

In [ ]:
obs_ap_gcrs2

In [ ]:
# extract degrees
ra_ap = obs_ap_gcrs2.ra.deg
dec_ap = obs_ap_gcrs2.dec.deg

# Report results
print(f'Difference in astromentric RA/DEC: Astropy (ICRS) vs. JPL\n')
diff_sf_sec = \
report_radec_diff(name1='JPL', name2='Astropy', ra1=ra_jpl, dec1=dec_jpl, ra2=ra_ap, dec2=dec_ap, obstime_mjd=obstime_mjd)

In [ ]:
dir(earth_bme)

### Appendix: dir call to Skyfield Classes

In [ ]:
type(earth)

In [ ]:
# dir(mars)
mars_dir = \
[ 'at',
 'center',
 'center_name',
 'ephemeris',
 'geometry_of',
 'negatives',
 'positives',
 'satellite',
 'target',
 'target_name',
 'topos']

In [ ]:
# dir(pos_mars)
pos_dir = \
['center',
 'cirs_radec',
 'cirs_xyz',
 'distance',
 'ecliptic_latlon',
 'ecliptic_position',
 'ecliptic_velocity',
 'ecliptic_xyz',
 'frame_xyz',
 'from_altaz',
 'galactic_latlon',
 'galactic_position',
 'galactic_xyz',
 'message',
 'observe',
 'observer_data',
 'position',
 'radec',
 'separation_from',
 'speed',
 't',
 'target',
 'to_skycoord',
 'velocity'] 